# Hämta bokningsbara tider
Anropar API:t och filtrerar fram eventuella bokningsbara tider mot covid-19.

## 1. Lista de platser som har lediga bokningsbara tider
Användningsfall: Lista information för den som är beredd att resa för att få sitt vaccin. Användaren får både reda på antalet lediga tider för stunden (till beslutet om hen kommer hinna boka en tid) samt en länk till där bokningen sker.

In [1]:
import requests
import json

# uncomment when the API is up and running again
"""
headers = {
    'client_id':'c4d279f9b8094dbaafd0162c5a606623', 
    'client_secret': '39D6cAB5D89c448ea3355aAC61De19e7'
}

req = requests.get('https://test.api.vgregion.se/e-crm-scheduling-public/api/v1/testCenter', headers = headers)
data = json.load(req.text)

testcenters = data['testcenters']

for testcenter in testcenters:
    if testcenter['timeslots'] != 0 and testcenter['timeslots'] != None:
        print('{0} har {1} st lediga tider - boka här {2}'.format(testcenter['title'], testcenter['timeslots'], testcenter['urlBooking']))
"""
  
# Opening JSON file
with open('../sample-data/vaccine-2021-05-25.json') as json_file:
    data = json.load(json_file)
    
    testcenters = data['testcenters']
  
    # Print the data of dictionary
    # print("\ntestcenters:", testcenters)

    # print(len(testcenters), 'testcenter')
    
    for testcenter in testcenters:
        if testcenter['timeslots'] != 0 and testcenter['timeslots'] != None:
            print('{0} har {1} st lediga tider - boka här {2}'.format(testcenter['title'], testcenter['timeslots'], testcenter['urlBooking']))

Närhälsan Lerums Gymnasium Häggen har 1 st lediga tider - boka här https://e-tjanster.1177.se/mvk/scheduling/tbMakeValidation.xhtml?hsaid=SE2321000131-E000000000431&dynamicid=bb0225fc-7c37-47c1-8558-ddb991501c2c
Närhälsan Bäckefors Dalslands Sjukhus har 1 st lediga tider - boka här https://e-tjanster.1177.se/mvk/scheduling/tbMakeValidation.xhtml?hsaid=SE2321000131-E000000002032&dynamicid=bb0225fc-7c37-47c1-8558-ddb991501c2c
Närhälsan Scandinavium har 2 st lediga tider - boka här https://e-tjanster.1177.se/mvk/scheduling/tbMakeValidation.xhtml?hsaid=SE2321000131-E000000016395&dynamicid=bb0225fc-7c37-47c1-8558-ddb991501c2c


## 2. Hur många tider finns i en viss kommun?
Användningsfall: Exempelvis en chatbot eller telefonservice. Svarar på frågan: 
> "Hur många vaccinationstider finns i [min-kommun]?"

In [2]:
import requests
import json
import csv

def initialize_municipalities():
    """
    Fetching the municipalities' names and codes
    """
    municipalities_dict = dict()
    
    with open('../sample-data/kommunkoder.csv', newline='') as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=';', quotechar='|')

        # adding municipalities and their corresponding codes to a dictionary
        for row in csv_reader:        
            municipalities_dict[row[1]] = row[0]

    return municipalities_dict

def initialize_testcenters():
    """
    Getting all available timeslots
    """
    testcenters = None
    
    with open('../sample-data/vaccine-2021-05-25.json') as json_file:
        data = json.load(json_file)

        testcenters = data['testcenters']
    
    return testcenters

def check_municipality_for_timeslots(municipalities, testcenters, municipality="Göteborg"):
    """
    Looking for eventual timeslots in a particular municipality
    """
    municipality_code = municipalities[municipality]
    
    timeslots = 0
    
    for testcenter in testcenters:
        if testcenter['municipality'] == municipality_code:
            if testcenter['timeslots'] != None:
                timeslots += int(testcenter['timeslots'])
    
    if timeslots > 0:
        return 'Det finns {0} lediga tider i {1}. Vill du boka en?'.format(timeslots, municipality)
    else:
        return 'Jag är ledsen! Det verkar inte finnas några bokningsbara vaccintider i {0} just nu.'.format(municipality)

# initializing external data resources
municipalities = initialize_municipalities()
testcenters = initialize_testcenters()

# testing some municipalities
print(check_municipality_for_timeslots(municipalities, testcenters, municipality="Göteborg"))
print(check_municipality_for_timeslots(municipalities, testcenters, municipality="Bengtsfors"))
print(check_municipality_for_timeslots(municipalities, testcenters, municipality="Mariestad"))

Det finns 2 lediga tider i Göteborg. Vill du boka en?
Det finns 1 lediga tider i Bengtsfors. Vill du boka en?
Jag är ledsen! Det verkar inte finnas några bokningsbara vaccintider i Mariestad just nu.
